<a href="https://colab.research.google.com/github/Dalthirr/NeuralNetworks/blob/EDA/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import necessary packages

In [0]:
import numpy as np
import pandas as pd
import sklearn
import keras
import tensorflow
import seaborn


Using TensorFlow backend.


In [0]:
print('test')

In [0]:
# Sołtysiak

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import  StandardScaler

df=pd.read_csv('C:\studia\Guass\Machine_Learning\Projekt_Mentor_Sebastian\Dane.csv')
print(df.head())
print(df.info())


# Wyrzucam wszystkie dane, które nie wpływają na pierwszy rzut oko nie wpływają na wynik
df=df.drop(['RowNumber','CustomerId','Surname'], axis=1)



print(df.head())
print(df.info())
StandardScaler(df)
print(df.describe())


#Proponuje zastosować dummy_variable dla kraju pochodzenia oraz płci.
df=pd.get_dummies(df,drop_first=True)

print(df.head())
print(df.info())

#Wypiszę wartości unikalne dla każdej zmiennej. Chcę dzięki temu wychwycić możliwe missing values
for i in range(len(df.columns)):
        print(df.columns[i])
        print(pd.unique(df.iloc[0:,i]))


      # Tworzę heatmapa korelacji pozostałych zmiennych. Odnoszę wrażenie, że są średnio skorelowane.
plt.show(sns.heatmap(df.corr(),xticklabels=df.corr().columns,
        yticklabels=df.corr().columns))





